In [1]:
Experiments_file='/home/kate/Research/YearBuilt/Experiments/Experiments.xlsx'
AllExperiments_tab='Experiments'
Experiment_name='Prediction'
#Experiment configuration:
#1.InputData is one line output frrom PreparingData trial. Only first line is used if more then 1 present
Experiment_InputData_tab='%s InputData'%Experiment_name
#2.ModelFiles: each line is a model name (Model) and full model file name (ModelData - model.tar.gz) in an S3 bucket
Experiment_ModelFiles_tab='%s ModelFiles'%Experiment_name

#Looks like Trial name should be unique in my environment, not in the experiment it belongs to
Trial_name='%s-Inference'%Experiment_name
 
bucket='kdproperty'
s3_batch_output = 's3://%s/Prediction/'%(bucket)

#number of instances can be teh same as number of features/files in InputData
instance_type='ml.m5.large'
instance_count=1

In [2]:
import boto3
import sys
import time
import pandas as pd
import numpy as np

import sagemaker
from sagemaker import get_execution_role
from sagemaker.xgboost.model import XGBoostModel

from sagemaker.analytics import ExperimentAnalytics

In [3]:
region = boto3.session.Session().region_name
role = 'arn:aws:iam::757107622481:role/service-role/AmazonSageMaker-ExecutionRole-20200819T131882'
sagemaker_session = sagemaker.session.Session(default_bucket=bucket)

'kdproperty'

In [4]:
sys.path.append('/home/kate/Research/YearBuilt/Notebooks/Property')
import ExperimentsUtils as eu

In [ ]:
#delete experiment and trials but not output files associated with jobs in experiments
#eu.cleanup_experiment(Experiment_name)
#eu.cleanup_trial(Experiment_name, Trial_name)

In [5]:
eu.create_experiment(Experiment_name)
eu.create_trial(Experiment_name,Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [6]:
#all models are predict from one dataset (in first row)
InputData = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_InputData_tab)
print(InputData['Data'][0])
s3_batch_input = InputData['Data'][0]

s3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-21-22-06-44-060/output/output


In [7]:
models_ModelFiles = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_ModelFiles_tab)
models_ModelFiles

,Model,Trial Component,ModelData
0,Construction_WaterScore_9,BestModels2-TrainingModels-Construction-WaterS...,s3://kdproperty/Construction-WaterScore-9-2020...
1,Construction_WaterScore_8,BestModels2-TrainingModels-Construction-WaterS...,s3://kdproperty/Construction-WaterScore-8-2020...


In [8]:
%%writefile inference.py
import json
import os
import pickle as pkl

import numpy as np

import sagemaker_xgboost_container.encoder as xgb_encoders


def model_fn(model_dir):
    """
    Deserialize and return fitted model.
    """
    model_file = "xgboost-model"
    booster = pkl.load(open(os.path.join(model_dir, model_file), "rb"))
    return booster


def input_fn(request_body, request_content_type):
    """
    The SageMaker XGBoost model server receives the request data body and the content type,
    and invokes the `input_fn`.

    Return a DMatrix (an object that can be passed to predict_fn).
    """
    if request_content_type == "text/csv":
        return xgb_encoders.csv_to_dmatrix(request_body.rstrip('\n').lstrip('\n'))
    else:
        raise ValueError(
            "Content type {} is not supported.".format(request_content_type)
        )


def predict_fn(input_data, model):
    """
    SageMaker XGBoost model server invokes `predict_fn` on the return value of `input_fn`.

    Return a two-dimensional NumPy array where the first columns are predictions
    and the remaining columns are the feature contributions (SHAP values) for that prediction.
    """
    prediction = model.predict(input_data)
    feature_contribs = model.predict(input_data, pred_contribs=True)
    output = np.hstack((prediction[:, np.newaxis], feature_contribs))
    
    return  prediction

Overwriting inference.py


In [9]:
#Models to be used in prediction
#based on model files provided in models_ModelFiles
models = list()
model_names = list()
i = 0
for index, row in models_ModelFiles.iterrows():
    #Try to delete if exists model and create a new model based on a model file
    name=row['Model']
    name=name.replace('_','-')
    model_data=row['ModelData']
    print(name,model_data)
    try:
        response = smclient.delete_model(ModelName=name)
        print('%s model was deleted'%name)
    except:
        print('%s model does not exist'%name)
        pass
    xgb_inference_model = XGBoostModel(
    name=name,
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    framework_version="1.0-1",
    )
    models.append(xgb_inference_model)
    model_names.append(name)
    print('%s model was created'%name)
    i = i + 1        

Construction-WaterScore-9 s3://kdproperty/Construction-WaterScore-9-2020-12-16-18-01-32/output/model.tar.gz
Construction-WaterScore-9 model does not exist
Construction-WaterScore-9 model was created
Construction-WaterScore-8 s3://kdproperty/Construction-WaterScore-8-2020-12-16-18-01-34/output/model.tar.gz
Construction-WaterScore-8 model does not exist
Construction-WaterScore-8 model was created


In [10]:
#create transformer job for each model
tranform_jobs = list()
tranformers = list()
i = 0
for m,model_name in zip(models,model_names):       
    s3_batch_output_model=s3_batch_output+model_name
    print(model_name,s3_batch_output_model)
    transformer =  m.transformer(
                                              instance_count=instance_count, 
                                              instance_type=instance_type,
                                              output_path=s3_batch_output_model,
                                              accept='text/csv',
                                              strategy='MultiRecord',
                                              assemble_with='Line'
                                             )
    tranformers.append(transformer)
    transformer.transform(data=s3_batch_input, content_type='text/csv',split_type='Line',
    experiment_config = {
        'ExperimentName': Experiment_name ,
        'TrialName' : Trial_name,
        'TrialComponentDisplayName' : '%s-%s'%(Trial_name,model_name.replace('_','-')),})
    job_name = transformer.latest_transform_job.name
    tranform_jobs.append(job_name)
    print('Job %s started'%job_name)
    i = i + 1

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Construction-WaterScore-9 s3://kdproperty/PartialDependency/Construction-WaterScore-9


INFO:sagemaker:Creating model with name: Construction-WaterScore-9
INFO:sagemaker:Creating transform job with name: Construction-WaterScore-9-2020-12-21-22-36-04-133
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Job Construction-WaterScore-9-2020-12-21-22-36-04-133 started
Construction-WaterScore-8 s3://kdproperty/PartialDependency/Construction-WaterScore-8


INFO:sagemaker:Creating model with name: Construction-WaterScore-8
INFO:sagemaker:Creating transform job with name: Construction-WaterScore-8-2020-12-21-22-36-06-881


Job Construction-WaterScore-8-2020-12-21-22-36-06-881 started


In [12]:
eu.wait_transform_jobs(processors=tranformers,tranform_jobs=tranform_jobs,check_every_sec=10,print_every_n_output=6,wait_min=30)

Transforming job Construction-WaterScore-9-2020-12-21-22-36-04-133 status: InProgress
Transforming job Construction-WaterScore-8-2020-12-21-22-36-06-881 status: InProgress
Continue waiting...
Transforming job Construction-WaterScore-9-2020-12-21-22-36-04-133 status: InProgress
Transforming job Construction-WaterScore-8-2020-12-21-22-36-06-881 status: InProgress
Continue waiting...
Transforming job Construction-WaterScore-9-2020-12-21-22-36-04-133 status: InProgress
Transforming job Construction-WaterScore-8-2020-12-21-22-36-06-881 status: InProgress
Continue waiting...
Transforming job Construction-WaterScore-9-2020-12-21-22-36-04-133 status: InProgress
Transforming job Construction-WaterScore-8-2020-12-21-22-36-06-881 status: InProgress
Continue waiting...
Transforming job Construction-WaterScore-9-2020-12-21-22-36-04-133 status: InProgress
Transforming job Construction-WaterScore-8-2020-12-21-22-36-06-881 status: InProgress
Continue waiting...
Transforming job Construction-WaterScore

In [13]:
trial_component_analytics = ExperimentAnalytics(
    experiment_name=Experiment_name
)
trial_comp_ds = trial_component_analytics.dataframe()
trial_ds=trial_comp_ds[trial_comp_ds['DisplayName'].str.contains(Trial_name)].copy()
trial_ds['Model']=trial_ds['DisplayName'].str.replace(Trial_name+'-','')
trial_ds['Model']=trial_ds['Model'].str.replace('-','_')
#trial_ds

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [14]:
#Prediction Result files
PredictedData = trial_ds[['Model','DisplayName','SageMaker.TransformOutput - Value']]
PredictedData.columns=['Model','DisplayName','Data']
PredictedData

,Model,DisplayName,Data
0,Construction_WaterScore_9,PartialDependency-Inference-Construction-Water...,s3://kdproperty/PartialDependency/Construction...
1,Construction_WaterScore_8,PartialDependency-Inference-Construction-Water...,s3://kdproperty/PartialDependency/Construction...


In [15]:
#Saving into the Experiment log file testing data prediction files
eu.SaveToExperimentLog(Experiments_file, '%s PredictedData'%Experiment_name, PredictedData)